In [2]:
%pip install -q langchain langchain-gigachat transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 3.5 MB/s eta 0:00:00


In [7]:
import os

from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_gigachat import GigaChat
from langchain_text_splitters import RecursiveCharacterTextSplitter

from google.colab import userdata

In [4]:
filepath = "/content/long_texts/meeting_transcription.txt"

with open(filepath, "r", encoding="utf-8") as file:
  text = file.read()

len(text)

85234

In [5]:
llm = GigaChat(
    credentials=userdata.get("GIGACHAT_APIKEY"),
    scope=userdata.get("GIGACHAT_SCOPE"),
    model="gigachat:latest",
    profanity_check=False,
    verify_ssl_certs=False,
)

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20000, chunk_overlap=300, length_function=len
)

chunks = text_splitter.split_text(text)

len(chunks)

5

In [10]:
INITIAL_PROMPT = """"Ты - профессиональный ассистент для суммаризации аудио транскрипций.
Твоя задача - создать четкую, структурированную и информативную summary по куску аудиозаписи.

**ИСХОДНЫЙ ТЕКСТ**
{text}

**ИНСТРУКЦИИ ДЛЯ SUMMARY:**

1. **ТИП SUMMARY:** Коспект в формате ключевые идеи + выводы
2. **Структура:**
   - **Основная тема:** 2-3 предложения
   - **Ключевые моменты:** 3-5 самых важных идей
   - **Конкретные примеры/факты:** 2-3 значимых детали
   - **Выводы/рекомендации:** что следует запомнить или сделать
   - **Имена и названия:** постарайся определить настоящие имена участников + оригинальные имена и названия обсуждаемого
   - **Цитирование:** цитаты для важных моментов по необходимости

3. **СТИЛЬ:** Профессионально, но доступно. Используй маркированные списки для наглядности.

**РЕЗУЛЬТАТ ДОЛЖЕН БЫТЬ ПОЛЕЗЕН ДЛЯ ТОГО, КТО НЕ СЛУШАЛ АУДИОЗАПИСЬ.**
"""

REFINE_PROMPT = """Твоя задача продолжить составление суммаризации аудио трансрибации совещания.

**ТЕКУЩАЯ СУММАРИЗАЦИЯ**
{summary}

**НОВАЯ ИНФОРМАЦИЯ**
{text}

**ИНСТРУКЦИИ ДЛЯ ПРОДОЛЖЕНИЯ SUMMARY**

1. **Интегрируй** новую информацию в существующую структуру.
2. **Не дублируй** уже имеющиеся сведения
3. **Устрани противоречия**, если они возникли
4. **Сохрани связность** и логику изложения
5. **Добавь новые темы/разделы**, если они появились
6. **Улучши формулировки** для лучшей читаемости
7. **Имена и названия** постарайся определить настоящие имена участников + оригинальные имена и названия обсуждаемого
8. **Цитирование:** цитаты для важных моментов по необходимости

Особое внимание на:
- Связь между предыдущими и новыми идеями
- Хронологическую последовательность (если важна)
- Причинно-следственные связи

Верни ОБНОВЛЕННУЮ суммаризацию.
"""

In [11]:
initial_chain = ChatPromptTemplate.from_template(INITIAL_PROMPT) | llm | StrOutputParser()
refine_chain = ChatPromptTemplate.from_template(REFINE_PROMPT) | llm | StrOutputParser()

current_summary = ""
for i, chunk in tqdm(enumerate(chunks)):
  if i == 0:
    current_summary = initial_chain.invoke({"text": chunk})
  else:
    refined_summary = refine_chain.invoke({"summary": current_summary, "text": chunk})
    current_summary = refined_summary

5it [00:24,  4.84s/it]


In [12]:
MEETING_PROTOCOL_PROMPT = """Задача: На основе транскрибации (текстовой расшифровки) совещания составить официальный протокол в структурированном формате.

## Шаг 1. Вводные данные
 * Название компании: [вписать]

 * Вид документа: [например, "Протокол производственного совещания"]

 * Дата проведения: [дд.мм.гггг]

 * Место проведения: [если важно]

## Шаг 2. Участники
 * Председатель: [ФИО, способ участия (лично/онлайн)]

 * Секретарь: [ФИО]

 * Присутствовали:

 * * [ФИО] – лично/онлайн

 * * [ФИО] – лично/онлайн
(и т. д.)

## Шаг 3. Повестка дня
Перечислить основные вопросы, которые обсуждались:

1. [Тема 1]

2. [Тема 2]

3. [Прочие вопросы]

## Шаг 4. Обсуждение (СЛУШАЛИ)
Кратко изложить ключевые моменты выступлений:

 * [ФИО или "Всех присутствующих"]: [основные тезисы, мнения, предложения]

## Шаг 5. Решения (ПОСТАНОВИЛИ)
Оформить в виде таблицы (если нужно) или списка:

| № п/п | Содержание поручения | Ответственный исполнитель (ФИО) | Срок выполнения |
| 1.1 | [Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
| 1.2 |	[Суть задачи] |	[ФИО] |	[дд.мм.гггг] |
Или списком:

[Задача] – [Исполнитель], срок до [дата].

## Шаг 6. Подписи
Председатель: __________ / [ФИО]

Секретарь: __________ / [ФИО]

Дополнительные указания:
Сохранять официально-деловой стиль.

Избегать лишних деталей, оставлять только ключевые решения.

Если в обсуждении были разногласия – указать альтернативные мнения.

Даты и сроки проверять на актуальность.

Пример заполнения:
"1.1 Восстановить документы по проекту 'Алькор' – Кравцова Л.М., срок до 10.06.2025"

Формат вывода: Готовый протокол в формате, аналогичном образцу.


Транскрибация совещания:
{summary}
"""

In [13]:
chain = ChatPromptTemplate.from_template(MEETING_PROTOCOL_PROMPT) | llm | StrOutputParser()

meeting_protocol = chain.invoke({"summary": current_summary})

with open("Протокол_совещания.txt", "w", encoding="utf-8") as file:
  file.write(meeting_protocol)